In [2]:
import requests
import xml.etree.ElementTree as ET
import csv
import os

BASEX_URL = "http://localhost:8984/rest"
DATABASE_NAME = "customer_service_management"
USERNAME = "admin"
PASSWORD = "admin"


def fetch_xml_data():
    response = requests.get(
        f"{BASEX_URL}/{DATABASE_NAME}/documents.xml", auth=(USERNAME, PASSWORD))
    if response.status_code == 200:
        return ET.ElementTree(ET.fromstring(response.text)).getroot()
    else:
        print(f"Error fetching data: {response.status_code}")
        return None


def parse_xml_elements(xml_root):
    clients = []
    complaints = []
    membership_issues = []
    contract_breaches = []

    for client in xml_root.find('Clients'):
        clients.append({
            "ClientID": client.get("ClientID"),
            "Name": client.get("Name"),
            "Contact": client.get("Contact"),
            "Address": client.get("Address"),
            "AreaType": client.get("AreaType")
        })

    for complaint in xml_root.find('Complaints'):
        complaints.append({
            "ComplaintID": complaint.get("ComplaintID"),
            "ClientID": complaint.get("ClientID"),
            "Type": complaint.get("Type"),
            "Date": complaint.get("Date"),
            "Status": complaint.get("Status")
        })

    for issue in xml_root.find('MembershipIssues'):
        membership_issues.append({
            "IssueID": issue.get("IssueID"),
            "ClientID": issue.get("ClientID"),
            "IssueType": issue.get("IssueType"),
            "Date": issue.get("Date"),
            "Status": issue.get("Status")
        })

    for breach in xml_root.find('ContractBreaches'):
        contract_breaches.append({
            "BreachID": breach.get("BreachID"),
            "ClientID": breach.get("ClientID"),
            "Reason": breach.get("Reason"),
            "Date": breach.get("Date"),
            "Status": breach.get("Status")
        })

    return clients, complaints, membership_issues, contract_breaches


def limit_rows(data, limit):
    return data[:limit] if len(data) > limit else data


def merge_tables(clients, complaints, membership_issues, contract_breaches):
    merged_data = []


    for complaint in complaints:
        client = next(
            (c for c in clients if c["ClientID"] == complaint["ClientID"]), None)
        if client:
            merged_data.append({**complaint, **client})


    for issue in membership_issues:
        client = next(
            (c for c in clients if c["ClientID"] == issue["ClientID"]), None)
        if client:
            merged_data.append({**issue, **client})


    for breach in contract_breaches:
        client = next(
            (c for c in clients if c["ClientID"] == breach["ClientID"]), None)
        if client:
            merged_data.append({**breach, **client})

    return merged_data


def write_to_csv(merged_data, filename):
    if merged_data:

        header = [
            "ClientID", "Name", "Contact", "Address", "AreaType",
            "ComplaintID", "Type", "Date", "Status",
            "IssueID", "IssueType",
            "BreachID", "Reason"
        ]


        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=header)
            writer.writeheader()


            for row in merged_data:
                row_with_all_fields = {
                    key: row.get(key, None) for key in header}
                writer.writerow(row_with_all_fields)

        print(f"CSV file '{filename}' written successfully.")
    else:
        print("No data to write to CSV.")


def main():

    xml_root = fetch_xml_data()

    if xml_root:

        clients, complaints, membership_issues, contract_breaches = parse_xml_elements(
            xml_root)


        clients_to_merge = limit_rows(clients, 50)
        complaints_to_merge = limit_rows(complaints, 50)
        membership_issues_to_merge = limit_rows(membership_issues, 20)
        contract_breaches_to_merge = limit_rows(contract_breaches, 20)


        merged_data = merge_tables(clients_to_merge, complaints_to_merge,
                                   membership_issues_to_merge, contract_breaches_to_merge)


        write_to_csv(merged_data, 'merged_xml.csv')


if __name__ == "__main__":
    main()

CSV file 'merged_xml.csv' written successfully.
